In [10]:
# import pandas as pd
import wrds
from parameters import *

In [11]:
# File position
resources="code/wrds/"
results="../../data/pulled/"

In [12]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection(wrds_username='xuanhenghuang')

Loading library list...
Done


In [13]:
###################
# Compustat Block #
###################

""" 
at:  assets - total
pstkl: Preferred Stock Liquidating Value 
txditc: Deferred Taxes and Investment Tax Credit 
pstkrv: Preferred Stock Redemption Value
seq: Stockholders' Equity - Total
pstk:  Preferred/Preference Stock (Capital) - Total 
dlc: Debt in Current Liabilities - Total
dltt: Long-Term Debt - Total
indfmt='INDL': This condition filters the data based on the industry format being used.  
                'INDL' indicates industry such as manufacturing, retail, construction and other 
                commercial operations other than financial services
datafmt='STD': This condition filters the data to include only companies that report using 
                the standard data format.
popsrc='D' : This condition filters the data to include only companies that report 
                using domestic accounting standards.
consol='C' : This condition filters the data to only include consolidated financial statements
"""

comp = conn.raw_sql(f"""
                    select gvkey, datadate, at, pstkl, txditc,
                    pstkrv, seq, pstk, dlc, dltt
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate between '{start_date}' and '{end_date}'
                    """, date_cols=['datadate'])

In [14]:
comp.to_csv(results + 'wrds_comp.csv', index=False)

In [17]:
######################
# CRSP Block Monthly #
######################
# sql similar to crspmerge macro

""" 
exchcd: Exchange Code - code indicating the exchange on which a security is listed. 
        Normal exchange codes are respectively 1,2, and 3 for NYSE, AMEX and the Nasdaq Stock MarketSM.
            1	New York Stock Exchange
            2	American Stock Exchange
            3	The Nasdaq Stock Market(SM)
shrcd: Share Code - two-digit code describing the type of shares traded
ret: Holding Period Return
retx: Holding Period Return without Dividends
shrout: Number of Shares Outstanding
prc: Price
"""

crsp_m = conn.raw_sql(f"""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '{start_date}' and '{end_date}'
                      and b.exchcd between 1 and 3
                      """, date_cols=['date']) 

In [18]:
crsp_m.to_csv(results + 'wrds_crsp_m.csv', index=False)

In [19]:
# add delisting return
""" 
dlret: Delisting Return
dlstdt: Delisting Date
"""
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """, date_cols=['dlstdt'])

In [20]:
dlret.to_csv(results + 'wrds_dlret.csv', index=False)

In [21]:
#######################
# CCM Block           #
#######################
""" 
linktype: Link Type - 2-character code providing additional detail on the usage of the link data available
linkprim: Primary Link Marker
linkdt: First Effective Date of Link - date marking the first effective date of the current link
linkenddt: Last Effective Date of Link -  date marking the last date where the link is valid
"""
ccm=conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

In [22]:
ccm.to_csv(results + 'wrds_ccm.csv', index=False)

In [7]:
######################
# CRSP Block Daily   #
######################
# sql similar to crspmerge macro

""" 
exchcd: Exchange Code - code indicating the exchange on which a security is listed. 
        Normal exchange codes are respectively 1,2, and 3 for NYSE, AMEX and the Nasdaq Stock MarketSM.
            1	New York Stock Exchange
            2	American Stock Exchange
            3	The Nasdaq Stock Market(SM)
shrcd: Share Code - two-digit code describing the type of shares traded
ret: Holding Period Return
retx: Holding Period Return without Dividends
shrout: Number of Shares Outstanding
prc: Price
"""

# where a.date between '{start_date}' and '{end_date}'

crsp_d = conn.raw_sql(f"""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.dsf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1980' and '12/31/1990'
                      and b.exchcd between 1 and 3
                      """, date_cols=['date']) 

In [8]:
crsp_d.to_csv(results + 'wrds_crsp_d.csv', index=False)